---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    import pandas as pd
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)    

    # 1. Convert series of dates in string form to a dataframe with three columns of integers: Year, Month, Day. 
    import re
    pd.set_option("display.max_rows", 500)
    df = pd.Series.to_frame(df, name = 'original_text')

    # First, for each series value, match all text sequence that satisfy any one of these following regular expression 
    # exemplars:
    # (1) 04/20/2009; 04/20/09; 4/20/09; 4/3/09 
    # or, (2) Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 
    # or, (3) 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # or, (4) Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    # or, (5) Feb 2009; Sep 2009; Oct 2010
    # or, (6) 2009; 2010

    # The regex for these exemplars are:

    # (1) e.g. 04/20/2009; 04/20/09; 4/20/09; 4/3/09 

    # regex A = numeric formats for months = (01~09 or 1~9) or (10~12) = (0{0,1}[1-9]|1[012]) 
    # regex B = numeric format for days in a month =  (01~09 or 1~9) or (10~29) or (30~31) = (0{0,1}[1-9]|[12]\d|30|31)
    # regex C = numeric format for year in general or abbreviated form = (1~9999) or (00~99) = ([1-9]\d{0,3}|\d\d)

    # final regex = A/B/C 

    regex_A = "(0{0,1}[1-9]|1[012])"
    regex_B = "(0{0,1}[1-9]|[12]\d|30|31)"
    regex_C = "([1-9]\d{0,3}|\d\d)"
    final_regex = "(" + regex_A + "/" + regex_B + "/" + regex_C + ")"

    df['regex_dates'] = "Regex No Match"
    df['Year'] = 'year'
    df['Month'] = 'month'
    df['Day'] = 'day'
    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x])
        if type(match) != type(None):
            if  df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if  (len(match.groups()[3]) == 2) and (match.groups()[3][0] != '0'):
                df['Year'][x] = '19'+ match.groups()[3]
            if  (len(match.groups()[3]) == 2) and (match.groups()[3][0] == '0'):
                df['Year'][x] = '20' + match.groups()[3]
            if  (len(match.groups()[3]) == 4):
                df['Year'][x] = match.groups()[3]
            df['Month'][x] = match.groups()[1]
            df['Day'][x] = match.groups()[2]
        else:
            pass

    # (2) e.g.  Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 

    # regex D = word or abbreviation format for months = Jan or January or Feb or February or ... or Dec or December = 
    # (\bJan\b)|(\bJanuary\b)|(\bFeb\b)|(\bFebruary\b)|(\bMar\b)|(\bMarch\b)|(\bApr\b)|(\bApril\b)|(\bMay\b)|(\bJun\b)|
    # (\bJune\b)|(\bJul\b)|(\bJuly\b)|(\bAug\b)|(\bAugust\b)|(\bSep\b)|(\bSeptember\b)|(\bOct\b)|(\bOctober\b)|(\bNov\b)|
    # (\bNovember\b)|(\bDec\b)|(\bDecember\b)
    # * To make regex D case insensitive, state re.IGNORECASE as a parameter in re.search() or re.match() methods.

    # regex E = delimiter symbols between month, day, year = ('-' or '.' or ',' or ' ') or ('. ' or ', ') = 
    # ([.,]\s{1}|[-.,\s])

    # regex F = numeric format for year in four digits (i.e. year two thousand is represented as 2000 rather than 00)
    # = ([1-9]\d{3})

    # final regex = DEBEF
    regex_D = "(Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|\
              November|Dec|December)"
    regex_E = "(?:[.,]\s{1}|[-.,\s])"
    regex_F = "([1-9]\d{3})"
    final_regex = "(" + regex_D + regex_E + regex_B + regex_E + regex_F + ")"

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x], flags = re.IGNORECASE)
        if type(match) != type(None):
            if df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                df['Year'][x] = match.groups()[3]
            if df['Month'][x] == 'month':
                df['Month'][x] = match.groups()[1]
            if df['Day'][x] == 'day':    
                df['Day'][x] = match.groups()[2]
        else:
            pass

    # (3) e.g. 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009;
    # final regex = BEDEF
    final_regex = "(" + regex_B + regex_E + regex_D + regex_E + regex_F + ")"

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x], flags = re.IGNORECASE)
        if type(match) != type(None):
            if  df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                df['Year'][x] = match.groups()[3]
            if df['Month'][x] == 'month':
                df['Month'][x] = match.groups()[2]
            if df['Day'][x] == 'day':
                df['Day'][x] = match.groups()[1]
        else:
            pass

    # (4) e.g. Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009;
    # regex G = date suffixes: 'st', 'nd', 'rd',and 'th' = 'st' or 'nd' or 'th' = (st|nd|rd|th)

    # final regex = D\sBGEC
    regex_G = "(st|nd|rd|th)"
    final_regex = "(" + regex_D + "\s" + regex_B + regex_G + regex_E + regex_C + ")"

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x])
        if type(match) != type(None):
            if df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                df['Year'][x] = match.groups()[3]
            if df['Month'][x] == 'month':    
                df['Month'][x] = match.groups()[1]
            if df['Day'][x] == 'day':
                df['Day'][x] = match.groups()[2]
        else:
            pass

    # (5) e.g. Feb 2009; Sep 2009; Oct 2010;
    # final regex = D\sF Feb 2009; Sep 2009; Oct 2010
    final_regex = "(" + regex_D + regex_E + regex_F + ")"

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x])
        if type(match) != type(None):
            if df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                df['Year'][x] = match.groups()[2]
            if df['Month'][x] == 'month':
                df['Month'][x] = match.groups()[1]
            if df['Day'][x] == 'day':
                df['Day'][x] = 1
        else:
            pass

    # (6) For dates in format M-dd-yy (e.g. 4-13-82)
    # final regex = A-B-C
    final_regex = "(" + regex_A + "-" + regex_B + "-" + regex_C + ")"

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x])
        if type(match) != type(None):
            if df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                if (len(match.groups()[3]) == 2) and (match.groups()[3][0] != '0'):
                    df['Year'][x] = '19'+ match.groups()[3]
                if (len(match.groups()[3]) == 2) and (match.groups()[3][0] == '0'):
                    df['Year'][x] = '20' + match.groups()[3]
                if (len(match.groups()[3]) == 4):
                    df['Year'][x] = match.groups()[3]
            if df['Month'][x] == 'month':
                df['Month'][x] = match.groups()[1]
            if df['Day'][x] == 'day':
                df['Day'][x] = match.groups()[2]
        else:
            pass

    #(7) e.g. 2/1973, 03/1990
    # final regex = A/F
    final_regex = "(" + regex_A + '/' + regex_F + ')'

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x])
        if type(match) != type(None):
            if df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                if (len(match.groups()[2]) == 2) and (match.groups()[2][0] != '0'):
                    df['Year'][x] = '19'+ match.groups()[3]
                if (len(match.groups()[2]) == 2) and (match.groups()[2][0] == '0'):
                    df['Year'][x] = '20' + match.groups()[3]
                if (len(match.groups()[2]) == 4):
                    df['Year'][x] = match.groups()[2]
            if df['Month'][x] == 'month':
                df['Month'][x] = match.groups()[1]
            if df['Day'][x] == 'day':
                df['Day'][x] = '1'
        else:
            pass

    # (8) e.g. 2009; 2010
    # final regex = F
    final_regex = regex_F

    for x in df['original_text'].index:
        match = re.search(pattern = final_regex, string = df['original_text'][x])
        if (type(match) != type(None)):
            if df['regex_dates'][x] == "Regex No Match":
                df['regex_dates'][x] = match.group(0)
            if df['Year'][x] == 'year':
                df['Year'][x] = match.groups(0)[0]
            if df['Month'][x] == 'month':
                df['Month'][x] = '1'
            if df['Day'][x] == 'day':
                df['Day'][x] = '1'
        else:
            pass

    # For df['Month'], convert months in word form to number form. 
    df['Month'] = df['Month'].str.replace(pat = r'(\bJan\b|\bJanuary\b)', repl = '1', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bFeb\b|\bFebruary\b)', repl = '2', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bMar\b|\bMarch\b)', repl = '3', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bApr\b|\bApril\b)', repl = '4', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bMay\b|\bMay\b)', repl = '5', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bJun\b|\bJune\b)', repl = '6', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bJul\b|\bJuly\b)', repl = '7', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bAug\b|\bAugust\b)', repl = '8', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bSep\b|\bSeptember\b)', repl = '9', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bOct\b|\bOctober\b)', repl = '10', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bNov\b|\bNovember\b)', repl = '11', flags = re.IGNORECASE)
    df['Month'] = df['Month'].str.replace(pat = r'(\bDec\b|\bDecember\b)', repl = '12', flags = re.IGNORECASE)

    # Convert all 'Year', 'Month', and 'Day' column values to int. 
    df['Year'] = df['Year'].astype(dtype="int32")
    df['Month'] = df['Month'].astype(dtype="int32")
    df['Day'] = df['Day'].astype(dtype="int32")

    # 2. Rank dates in order of oldest to newest dates. Indicate the rank of the date by first labeling the date with its original
    # index in a new column, then move the values of this new column to the dataframe index which denotes its chronological rank. 

    # e.g. (index 0: 1999, index 1: 2001, index 2:  2000). Then first, (index 0 : 0, index 1: 1, index 2: 2), then rank the old 
    # index values by its date values in increasing order (index 0: 0, index 1: 2, index 2: 1)

    df['Original_index_position'] = df.index

    # Option (1): Add the elements of the original iterable to an empty list in succession, with the condition that when an element 
    # is added to the new list, sort the new list before adding another element from the old iterable. 

    # e.g. old_list [10, 5, 2, 78, 43, 9], new_list = []. Step 1: new_list = [10], new_list = new_list.sort(reverse = False) 
    # (default is reverse = False for sort by ascending order). Step 2: new_list = [10, 5], new_list = new_list.sort().
    # Step 3: new_list = [5, 10, 2], new_list = new_list.sort(). Step 4: new_list = [2,5,10,78], new_list = new_list.sort().
    # Step 5: new_list = [2,5,10,78,43], new_list = new_list.sort(). Step 6: new_list = [2,5,10,43,78,9], new_list= new_list.sort().
    # Then new_list = [2,5,9,10,43,78]

    # Applying this strategy, first group each row into a tuple which takes the value of each column as its element in order of 
    # the column's position in the dataframe. 
    # e.g. first row : (1993, 3, 25, 0), second row: (1985, 6, 18, 1), third row: (1971, 7, 8, 2), etc..
    # These tuples are the elements that I will sort in increasing order of their ordinal values (the first three values of the 
    # tuple). 

    old_list = list(map(tuple, df[['Year', 'Month', 'Day', 'Original_index_position']].values))

    new_list = sorted(old_list) 
                    # The sort() method by default, sorts elements of an iterable in ascending order. If the elements are vectors, 
                    # by default the vector elements are ordered by comparing the elements of the vectors of the same
                    # dimension, with descending order of priority of sorting from left-most dimension of the vector to the 
                    # right-most dimension of the vector. I.e. for vector elements, the first elements of each vector is compared,
                    # and if and only if there is a tie between two n th dimension elements of two vectors, the two n+1 th elements 
                    # of the same two vectors are compared to determine the order. 

    # Update the 'Original_index_position' column to map the rank of the date to the date corresponding to
    # the original index position of the dataframe. 
    ranks = []

    for x in new_list:
        ranks.append(x[3])    

    df['Original_index_position'] = ranks

    return pd.Series(df['Original_index_position'], dtype="int32")